In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

### SERTI (EXTRAIDO EN EXCEL)

In [ ]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\10. OCTUBRE\SERTI\*.csv',recursive=False)
serti_files


In [ ]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])                     
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

In [ ]:
serti.dtypes

In [ ]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

In [ ]:
duplicates_serti.shape

In [ ]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [ ]:
serti.shape

In [ ]:
serti.dtypes

### DB PRODUCTO

In [ ]:
producto_files_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\10. OCTUBRE\DB\TOPUPS DB FACTURA 2021.csv',
                              dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
                                     'status_code':object})
# producto_files_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\2021\11. NOVIEMBRE\DB\TOPUPS NOVIEMBRE 2021.csv',
#                               dtype={'clip_external_tracking_id':object,'phone_number':object,'last4':object,'amount':float,
#                                      'status_code':object})


In [ ]:
# topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'phone_number':object,
#                                             'last4':object,'amount':float,'status_code':object})])
topups_db = pd.concat([producto_files_1])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['clip_external_tracking_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db.head()

In [ ]:
duplicates = topups_db[topups_db.duplicated(['clip_external_tracking_id'], keep=False)]
duplicates.head()

In [ ]:
duplicates.shape

In [ ]:
duplicates['clip_external_tracking_id'].unique()

In [ ]:
topups_db.shape

In [ ]:
topups_db.dtypes

### PIVOT DB

In [ ]:
pivot_db = pd.pivot_table(topups_db,index=['updated_at']
                            ,columns=['status_code']
                            ,values=['amount','clip_external_tracking_id']
                            ,aggfunc={'amount':np.sum,
                                     'clip_external_tracking_id':len},margins=True).reset_index()
pivot_db.fillna(0,inplace=True)
pivot_db.head()

In [ ]:
pivot_db.shape

### DB VS SERTI

In [ ]:
cruce_1 = topups_db.merge(serti[['INDEX_TOTAL','transactionId','panTerminacion','last4_phone_serti','serti_date','importe'
                                 ,'aprobada','carrier']]
                      ,on='INDEX_TOTAL',how='left')
cruce_1['aprobada'].fillna('Not_found',inplace=True)
cruce_1['transactionId'].fillna('Not_found',inplace=True)
cruce_1.head()

In [ ]:
cruce_1.shape

### STATUS 1 ONLY

In [ ]:
cruce_1_status_1 = cruce_1[(cruce_1['status_code'] == '1')]
cruce_1_status_1['importe'].fillna(0,inplace=True)
cruce_1_status_1.head()

In [ ]:
cruce_1_status_1.shape

In [ ]:
duplicates_cruce_1_status_1 = cruce_1_status_1[cruce_1_status_1.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_status_1.head()

In [ ]:
duplicates_cruce_1_status_1.shape

In [ ]:
cruce_1_final = cruce_1_status_1[(cruce_1_status_1['transactionId'] != 'Not_found')]
cruce_1_final.head()

In [ ]:
cruce_1_final.shape

In [ ]:
na_cruce_1_final = cruce_1_status_1[(cruce_1_status_1['transactionId'] == 'Not_found')]
na_cruce_1_final.head()

In [ ]:
na_cruce_1_final.shape

In [ ]:
cruce_1_status_1.columns

### PIVOT STATUS 1 ONLY

In [ ]:
pivot_status_1 = pd.pivot_table(cruce_1_status_1,index=['updated_at']
                            ,columns=['aprobada']
                            ,values=['importe','amount','transactionId','clip_external_tracking_id']
                            ,aggfunc={'importe':np.sum,
                                     'amount':np.sum,
                                     'transactionId':len,
                                     'clip_external_tracking_id':len},margins=True).reset_index()
pivot_status_1.fillna(0,inplace=True)
pivot_status_1.head()

In [ ]:
pivot_status_1.shape

# EXCEL 

In [ ]:
writer = pd.ExcelWriter('CLIP-SERTI TOPUPS OCTUBRE 2021.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
pivot_db.to_excel(writer, sheet_name='RESUMEN DB')
pivot_status_1.to_excel(writer, sheet_name='RESUMEN')
cruce_1_status_1.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_status_1.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.save()
writer.close()